In [10]:
# !pip install langchain
# !pip install -q langchain-community langchain_huggingface chromadb --use-feature=fast-deps
# !pip install lxml

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
import requests

# SKT 한국어 임베딩 및 QA 모델 설정
hf_embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

c:\Users\zqrc0\anaconda3\envs\dlblm\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [1]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

os.environ.get("HUGGINGFACEHUB_API_TOKEN")

''

In [12]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
from bs4 import BeautifulSoup as bs

# 검색부
def load_Korean_game(gametitle, display=5, pageno=1):
    url = f'https://www.grac.or.kr/WebService/GameSearchSvc.asmx/game?display={display}&pageno={pageno}&gametitle={gametitle}'
    
    response = requests.get(url)
    response.raise_for_status()  # 오류 발생 시 예외 처리

    # HTML 파싱 및 본문 내용 추출
    soup = bs(response.text, 'html.parser')
    content = soup.find_all('item')  # item 태그들 추출
    
    paragraphs = []
    for idx, i in enumerate(content):
        # print(f'idx : {idx}')
        # print('-'*50)
        # print(f'content : {i}')
        # title = i.find('gametitle')  # gametitle 태그 추출
        if i:
            # print(f'gametitle : {i.text}')
            paragraphs.append(i.text)
    
    # 텍스트 변환 및 Document 객체로 변환
    text = "\n".join(paragraphs)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    # 텍스트 스플릿
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

def create_vectorstore(splits):
    valid_splits = []
    for i, split in enumerate(splits):
        print(f"Processing split {i+1}/{len(splits)}: {split.page_content[:30]}...")  # 첫 30자 출력
        
        # 임베딩 생성 과정 확인
        embedding = hf_embeddings.embed_query(split.page_content)
        
        # 임베딩이 올바르게 생성되었는지 길이와 형태 출력
        if embedding and len(embedding) > 0:
            print(f"Embedding created for split {i+1}, length: {len(embedding)}")
            valid_splits.append(split)
        else:
            print(f"Empty embedding for split {i+1}")

    if valid_splits:
        vectorstore = Chroma.from_documents(documents=valid_splits, embedding=hf_embeddings)
        return vectorstore
    else:
        raise ValueError("No valid documents with embeddings to add to vector store.")

In [13]:
from langchain_community.vectorstores import Chroma

# 사용 예시
topic = '화이트데이'
splits = load_Korean_game(topic)
if splits:
    vectorstore = create_vectorstore(splits)
else:
    print("No content to create vector store.")

c:\Users\zqrc0\anaconda3\envs\dlblm\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Processing split 1/1: GC-CC-NP-230203-016
2023-02-03...
Embedding created for split 1, length: 768


In [14]:
from transformers import pipeline

# KoAlpaca 모델 로드
qa_pipeline = pipeline(
    "text2text-generation",
    model="beomi/KoAlpaca-Polyglot-5.8B",
    tokenizer="beomi/KoAlpaca-Polyglot-5.8B"
)

Loading checkpoint shards: 100%|██████████| 13/13 [00:46<00:00,  3.57s/it]
The model 'GPTNeoXForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditio

In [15]:
# context 구성
context = " ".join([split.page_content for split in splits])

# 질문 설정
question = "화이트데이 스토리 알려줘"

# context와 question을 하나의 문자열로 합쳐서 전달
input_text = f"{context}\n\n질문: {question}"
result = qa_pipeline(input_text, max_new_tokens=100)

# 질문과 답변만 출력
print("질문:", question)
print("답변:", result[0]['generated_text'])

질문: 화이트데이 스토리 알려줘
답변: GC-CC-NP-230203-016
2023-02-03
화이트데이2: 거짓말하는 꽃(White Day2: The Flower That Tells Lies) (PC)
게임콘텐츠등급분류위원회
주식회사 루트엔스튜디오
화이트데이 전날 밤 사고로 폐쇄된 학교에서 숨겨진 진실을 찾는 공포 어드벤처 게임
15세이용가
어드벤처
PC/온라인 게임
공포
False



GC-CC-NP-180126-010
2018-01-26
화이트데이: 담력시험
게임콘텐츠등급분류위원회
주식회사 손노리
학교의 폐쇠된 창고에서 진행하는 담력시험 게임으로 지시된 아이템들을 제한시간 내에 찾아오는 VR기기 전용 PC용 공포액션게임
12세이용가
액션
PC/온라인 게임
공포
False



GC-CC-NV-170203-005
2017-02-03
화이트데이: 학교라는 이름의 미궁
게임콘텐츠등급분류위원회
주식회사 손노리
화이트데이 전날 밤의 학교를 배경으로 숨겨진 사연에 접근해 나가는 어드벤처 게임.
15세이용가
어드벤처
비디오 게임
공포
False



GC-CC-NP-160513-001
2016-05-13
화이트데이 : 학교라는 이름의 미궁
게임콘텐츠등급분류위원회
주식회사 손노리
화이트데이 전날 밤의 학교를 배경으로 숨겨진 사연에 접근해 나가는 어드벤처 게임
15세이용가
어드벤처
PC/온라인 게임
공포
False



CC-NA-120718-009
2012-07-18
White Day Triple (화이트데이트리플)
게임물관리위원회
(주)한국오토엠
로봇팔을 이용한 크레인 게임, 바가지 형태의 로봇팔과 움직이는 밀판을 이용하여 경품을 획득 할 수 있는 게임물 

전체이용가
기타
아케이드 게임

False

질문: 화이트데이 스토리 알려줘

### 맥락:
화이트데이는 3월 14일이죠. 성 발렌타인 축일에 얽힌 사연에 맞춰서 온갖 소문이 떠돌던데(예를 들면 성 발렌타인을 사모하던 여인이 죽은 날이라든가) 그냥 만들어낸 이야기로 보면 맞습니다.정확히 3월 14일이 기념일 아